In [ ]:
# ========== 1. 环境与依赖 ==========
import os, json, warnings
warnings.simplefilter("ignore", FutureWarning)


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.patches as mpatches
import shap, math
import matplotlib as mpl


from matplotlib import rcParams
from IPython.display import display
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold, cross_val_score, StratifiedKFold, GridSearchCV, learning_curve
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score, recall_score, confusion_matrix, precision_score, make_scorer
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from boruta import BorutaPy
from scipy.stats import gaussian_kde
from sklearn.utils import check_random_state
from matplotlib.lines import Line2D
from sklearn.pipeline import Pipeline
from sklearn.calibration import CalibratedClassifierCV
from xgboost import XGBClassifier
from sklearn.impute import SimpleImputer  
from sklearn.base import clone
from tqdm import tqdm
from scipy.special import logit, expit
from numpy.random import default_rng
from sklearn.calibration import CalibratedClassifierCV, calibration_curve
from sklearn.metrics import brier_score_loss
from sklearn.feature_selection import mutual_info_classif
from sklearn.linear_model import Lasso
from sklearn.metrics import log_loss, f1_score


# 统一使用一个随机种子
RANDOM_STATE = 2025
np.random.seed(RANDOM_STATE)

def setup_chinese_fonts():
    """
    配置matplotlib支持中文显示
    """
    # 方案1：使用系统中文字体（推荐）
    try:
        rcParams['font.sans-serif'] = ['SimHei', 'DejaVu Sans', 'Arial Unicode MS', 'Microsoft YaHei']
        rcParams['axes.unicode_minus'] = False  # 解决负号显示问题
        print("✅ 中文字体配置成功")
    except Exception as e:
        print(f"⚠️ 中文字体配置失败: {e}")
        print("   使用英文标签作为备选方案")

# 立即配置
setup_chinese_fonts()

In [ ]:
# ========== 2. 读取数据 ==========
DATA_PATH = r"C:/Users/VVai/Desktop/output/清洗/整合数据整理/output_with_CCI_total.csv"

TARGET = "death_flag"

df = pd.read_csv(DATA_PATH)
print("数据形状:", df.shape)
print("列名(预览)前30个:", list(df.columns)[:30])
assert TARGET in df.columns, f"未找到目标变量列 {TARGET}"

# 候选特征
candidate_cols = [
    # 基本信息
    "age", "gender",

    # 体格信息
    "Height", "Height (Inches)", "Weight", "Weight (Lbs)", "BMI", "BMI (kg/m2)",

    # 血压
    "Diastolic blood pressure", "Systolic blood pressure",

    # 甲功相关
    "PTFQI",

    # 血液/生化指标
    "RBC(m/uL)", "WBC(K/uL)", "PLT", "Hb(g/dL)", "Glucose(mg/dL)",
    "Creatinine(mg/dL)", "BUN(mg/dL)", "ALT(IU/L)", "AST", "PT",

    # 疾病/并发症
    "Hypertension", "Diabetes", "CKD", "Myocardial infarct",
    "Congestive heart failure", "Malignant cancer", "COPD",

    # 住院信息（根据目标变量调整）
    "CCI Score",
]

#从特征中排除目标变量
feature_cols = [c for c in candidate_cols if c in df.columns and c != TARGET]

# 1：确保目标变量不在特征中
print("\n" + "=" * 60)
print("🔍 安全检查")
print("=" * 60)
print(f"目标变量: {TARGET}")
print(f"候选特征数量: {len(feature_cols)}")

if TARGET in feature_cols:
    print("严重错误：目标变量出现在特征列表中！")
    print("这会导致数据泄露，结果完全不可信！")
    raise ValueError(f"目标变量 {TARGET} 不能作为特征！")
else:
    print(f"通过：目标变量 '{TARGET}' 未出现在特征中")

# 2：打印特征列表供人工审核
print(f"\n用于建模的特征列表 ({len(feature_cols)} 个):")
for i, col in enumerate(feature_cols, 1):
    print(f"  {i:2d}. {col}")

# 性别映射(如是字符串)
if "gender" in feature_cols and df["gender"].dtype == object:
    df["gender"] = df["gender"].map({"M":1, "F":0}).fillna(0).astype(int)
    print("\n 性别已映射: M → 1, F → 0")

# 构建建模数据集
df_model = df[feature_cols + [TARGET]].dropna().copy()
X_all = df_model[feature_cols]
y_all = df_model[TARGET].astype(int)

# 最终验证
print("\n" + "=" * 60)
print("✅ 数据准备完成")
print("=" * 60)
print(f"样本数量: {X_all.shape[0]}")
print(f"特征数量: {X_all.shape[1]}")
print(f"目标变量: {TARGET}")
print(f"\n目标变量分布:")
print(y_all.value_counts())
print(f"正类比例: {y_all.mean():.2%}")

# 确认 X_all 中没有目标变量
if TARGET in X_all.columns:
    raise ValueError(f" 致命错误：目标变量 {TARGET} 在特征矩阵中！")
print(f"\n 最终确认：特征矩阵中不含目标变量")
print("=" * 60)

# 在第2步末尾添加
print(f"\n 类别分布分析:")
print(f"  负类(存活): {(y_all==0).sum()} ({(y_all==0).mean():.1%})")
print(f"  正类(死亡): {(y_all==1).sum()} ({(y_all==1).mean():.1%})")
print(f"  不平衡比: 1:{(y_all==0).sum()/(y_all==1).sum():.1f}")

In [ ]:
# ============================ 第3步:数据集划分 ============================
# Step 1: 先划分出测试集(10%)
X_temp, X_test, y_temp, y_test = train_test_split(
    X_all, y_all,
    test_size=0.1,
    random_state=RANDOM_STATE,
    stratify=y_all  # 保证类别比例一致
)

# Step 2: 剩下的 90% 再按 8:1 划分训练集与验证集
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp,
    test_size=1/9,   # 90% 的 1/9 ≈ 10%
    random_state=RANDOM_STATE,
    stratify=y_temp
)

# 保存副本(用于归一化 / 特征选择)
train_y = y_train.copy()
val_y = y_val.copy()
test_y = y_test.copy()


print("数据集划分完毕:")
print(f"训练集:{X_train.shape}, 验证集:{X_val.shape}, 测试集:{X_test.shape}")

In [ ]:
# ============================ 第4步:数据预处理 ============================

# 直接使用最终变量名，避免中间变量
train_X_scaled = X_train.copy()
val_X_scaled = X_val.copy()
test_X_scaled = X_test.copy()

print("=" * 70)
print("开始数据预处理")
print("=" * 70)
print(f"训练集: {train_X_scaled.shape}")
print(f"验证集: {val_X_scaled.shape}")
print(f"测试集: {test_X_scaled.shape}")

# 识别变量类型
binary_cols = []
cont_cols = []

for col in train_X_scaled.columns:
    unique_vals = train_X_scaled[col].dropna().unique()
    if set(unique_vals).issubset({0, 1}) or set(unique_vals).issubset({0.0, 1.0}):
        binary_cols.append(col)
    else:
        cont_cols.append(col)

print(f"\n二分类变量 ({len(binary_cols)}个): {binary_cols}")
print(f"连续变量 ({len(cont_cols)}个): {cont_cols}")

# 缺失值填充
if len(cont_cols) > 0:
    imputer_cont = SimpleImputer(strategy='median')
    train_X_scaled[cont_cols] = imputer_cont.fit_transform(train_X_scaled[cont_cols])
    val_X_scaled[cont_cols] = imputer_cont.transform(val_X_scaled[cont_cols])
    test_X_scaled[cont_cols] = imputer_cont.transform(test_X_scaled[cont_cols])
    print(f" 连续变量缺失值填充完成 (中位数)")

if len(binary_cols) > 0:
    imputer_binary = SimpleImputer(strategy='most_frequent')
    train_X_scaled[binary_cols] = imputer_binary.fit_transform(train_X_scaled[binary_cols])
    val_X_scaled[binary_cols] = imputer_binary.transform(val_X_scaled[binary_cols])
    test_X_scaled[binary_cols] = imputer_binary.transform(test_X_scaled[binary_cols])
    print(f" 二分类变量缺失值填充完成 (众数)")

# 偏斜处理
skew_threshold = 2.0
highly_skewed = []
for col in cont_cols:
    skewness = train_X_scaled[col].skew()
    if abs(skewness) > skew_threshold and train_X_scaled[col].min() >= 0:
        highly_skewed.append(col)

if len(highly_skewed) > 0:
    print(f"\n检测到 {len(highly_skewed)} 个高偏斜变量: {highly_skewed}")
    train_X_scaled[highly_skewed] = np.log1p(train_X_scaled[highly_skewed])
    val_X_scaled[highly_skewed] = np.log1p(val_X_scaled[highly_skewed])
    test_X_scaled[highly_skewed] = np.log1p(test_X_scaled[highly_skewed])
    print(f" 已进行 log1p 转换")

# 标准化 (仅对连续变量)
scaler = RobustScaler()
train_X_scaled[cont_cols] = scaler.fit_transform(train_X_scaled[cont_cols])
val_X_scaled[cont_cols] = scaler.transform(val_X_scaled[cont_cols])
test_X_scaled[cont_cols] = scaler.transform(test_X_scaled[cont_cols])

print(f"\n 预处理完成")
print(f"   - 缺失值填充: ")
print(f"   - 偏斜处理: {len(highly_skewed)} 个变量")
print(f"   - 标准化: {len(cont_cols)} 个连续变量")
print("=" * 70)

# 创建变量别名，供后续可视化使用
train_X_imp = train_X_scaled.copy()
val_X_imp = val_X_scaled.copy()
test_X_imp = test_X_scaled.copy()

print(f" 已创建预处理后的数据副本: train_X_imp, val_X_imp, test_X_imp")

In [ ]:
# ============================ 第5步:Boruta 特征选择 ============================

print("=" * 50)
print("开始 Boruta 特征选择")
print("=" * 50)

# Boruta 参数设置
selector = BorutaPy(
    RandomForestClassifier(
        n_jobs=-1, 
        class_weight='balanced', 
        max_depth=5,
        random_state=RANDOM_STATE
    ),
    n_estimators='auto',
    verbose=2,
    random_state=RANDOM_STATE
)

# 在训练集上进行特征选择
selector.fit(train_X_scaled.values, train_y.values)

# 获取选中的特征
selected_features = train_X_scaled.columns[selector.support_].tolist()
tentative_features = train_X_scaled.columns[selector.support_weak_].tolist()
selected_all = selected_features + tentative_features

print("\n" + "=" * 50)
print(f" Boruta 特征选择完成")
print("=" * 50)
print(f"确认选中的特征 ({len(selected_features)}): {selected_features}")
print(f"待定特征 ({len(tentative_features)}): {tentative_features}")
print(f"最终使用的特征总数 ({len(selected_all)}): {selected_all}")

# 保存特征选择结果
feature_importance = pd.DataFrame({
    'Feature': train_X_scaled.columns,
    'Selected': selector.support_,
    'Tentative': selector.support_weak_,
    'Ranking': selector.ranking_
}).sort_values('Ranking')

print("\n特征重要性排名:")
print(feature_importance.to_string(index=False))

print("\n" + "=" * 50)
print("应用特征选择到所有数据集")
print("=" * 50)

train_X_sel = train_X_scaled[selected_all].copy()
val_X_sel = val_X_scaled[selected_all].copy()
test_X_sel = test_X_scaled[selected_all].copy()

# 统一变量名，供后续步骤使用
X_train_for_models = train_X_sel
X_val_for_models = val_X_sel
X_test_for_models = test_X_sel

print(f"训练集: {train_X_sel.shape}")
print(f"验证集: {val_X_sel.shape}")
print(f"测试集: {test_X_sel.shape}")

In [ ]:
# ===================== Boruta 内部机制可视化（标准Boruta方式：vs. Shadow Max）=====================
# ---------- 可配置 ----------
N_ITERS = 200
SAMPLE_FRAC = 0.8
SEED = 42

# ----------  使用建模时相同的特征集 ----------
X_sel = train_X_scaled[selected_all].copy()
y_sel = train_y.copy()

print("=" * 70)
print("🎨 Boruta 山脊图可视化（标准Boruta方式：vs. Shadow Max）")
print("=" * 70)
print(f"可视化特征数: {len(selected_all)}")
print(f"  - 确认特征: {len(selected_features)}")
print(f"  - 待定特征: {len(tentative_features)}")
print(f"迭代次数: {N_ITERS}")
print("=" * 70)

# ---------- RF 参数 ----------
if 'selector' in globals():
    RF_PARAMS = selector.estimator.get_params()
    RF_PARAMS.pop('random_state', None)
else:
    RF_PARAMS = dict(
        n_estimators=300,
        n_jobs=-1,
        max_depth=5,
        class_weight='balanced'
    )

# ---------- 生成标准Boruta Z-score分布（与Shadow Max比较） ----------
def collect_boruta_z_vs_shadow_max(X_df, y, rf_params, n_iters=200, sample_frac=0.8, seed=42):
    rng = default_rng(seed)
    cols = list(X_df.columns)
    n = len(X_df)
    take = max(1, int(n * float(sample_frac)))
    rows = []

    base_params = dict(rf_params)
    base_params.pop("random_state", None)

    for i in tqdm(range(n_iters), desc="Collecting Boruta Z (vs. Shadow Max)"):
        idx = rng.choice(n, take, replace=True)
        Xb = X_df.iloc[idx, :]
        yb = y.iloc[idx]

        X_shadow = pd.DataFrame(index=Xb.index)
        for c in cols:
            shuf = Xb[c].to_numpy().copy()
            rng.shuffle(shuf)
            X_shadow[c + "_shadow"] = shuf

        X_aug = pd.concat([Xb.reset_index(drop=True), X_shadow.reset_index(drop=True)], axis=1)
        
        rf_local = RandomForestClassifier(**base_params, random_state=int(seed) + i)
        rf_local.fit(X_aug, yb.values)

        imps = pd.Series(rf_local.feature_importances_, index=X_aug.columns)
        imp_real   = imps.loc[cols].values
        imp_shadow = imps.loc[[c + "_shadow" for c in cols]].values

        # 使用shadow最大值（MZSA）
        sh_max = float(np.max(imp_shadow))
        sh_std = float(np.std(imp_shadow))
        
        if sh_std == 0 or np.isnan(sh_std):
            sh_std = 1e-12
        
        z = (imp_real - sh_max) / sh_std

        rows.extend([(f, float(zv), i) for f, zv in zip(cols, z)])

    long_df = pd.DataFrame(rows, columns=["feature", "z", "iter"])
    return long_df

long_borutaZ = collect_boruta_z_vs_shadow_max(
    X_df=X_sel, y=y_sel,
    rf_params=RF_PARAMS,
    n_iters=N_ITERS, sample_frac=SAMPLE_FRAC, seed=SEED
)

# ---------- 合并 Boruta 判定结果（用于着色） ----------
decision_map = {}
if 'selector' in globals(): 
    confirmed_feats = set(train_X_scaled.columns[np.where(selector.support_)[0]])  
    tentative_feats_set = set(train_X_scaled.columns[np.where(selector.support_weak_)[0]])
    for f in X_sel.columns:
        if f in confirmed_feats:
            decision_map[f] = "Confirmed"
        elif f in tentative_feats_set:
            decision_map[f] = "Tentative"
        else:
            decision_map[f] = "Rejected"
else:
    decision_map = {f: "Tentative" for f in X_sel.columns}

long_borutaZ["decision"] = long_borutaZ["feature"].map(decision_map)

# ---------- 按中位数 Z 排序 ----------
feat_order = (long_borutaZ.groupby("feature")["z"]
              .median().sort_values(ascending=False).index.tolist())
feats_for_plot = feat_order[::-1]

# ---------- Ridge 山脊图函数 ----------
def ridgeline_plot_borutaZ(df_long, feats,
                           x_label="Boruta Z-score (vs. Shadow Max)",
                           title="Feature Importance vs. Shadow Maximum (Standard Boruta)"):
    style_map = {
        'Confirmed': {'lw': 1.8, 'alpha': 0.9},
        'Tentative': {'lw': 1.2, 'alpha': 0.75},
        'Rejected' : {'lw': 1.0, 'alpha': 0.55},
    }
    color_map = {
        'Confirmed': '#1f77b4',
        'Tentative': '#9467bd',
        'Rejected' : '#7f7f7f',
    }

    all_z = df_long['z'].dropna().values
    if len(all_z):
        x_min = float(np.percentile(all_z, 0.5) - 0.5)
        x_max = float(np.percentile(all_z, 99.5) + 0.5)
    else:
        x_min, x_max = -1.0, 1.0
    xs = np.linspace(x_min, x_max, 350)

    fig, ax = plt.subplots(figsize=(8.5, max(4.0, 0.44*len(feats)+2)))
    y_offsets = np.arange(len(feats))

    for y0, f in zip(y_offsets, feats):
        sub = df_long.loc[df_long['feature'] == f]
        dat = sub['z'].dropna().values
        main_dec = sub['decision'].mode().iat[0] if len(sub) and not sub['decision'].mode().empty else 'Tentative'
        style = style_map.get(main_dec, style_map['Tentative'])
        c = color_map.get(main_dec, color_map['Tentative'])

        if len(dat) < 5 or np.allclose(dat, dat[0]):
            xm = float(np.mean(dat)) if len(dat) else 0.0
            ax.plot([xm, xm], [y0, y0+0.85], lw=style['lw'], alpha=style['alpha'], color=c)
            continue

        kde = gaussian_kde(dat)
        ys = kde(xs)
        ys = ys / (ys.max() + 1e-12) * 0.8

        ax.fill_between(xs, y0, y0 + ys, alpha=style['alpha'], color=c)
        ax.plot(xs, y0 + ys, lw=style['lw'], color=c)
        ax.plot(xs, [y0]*len(xs), lw=0.6, color='white')

    ax.set_xlabel(x_label)
    ax.set_yticks(y_offsets + 0.4)
    ax.set_yticklabels(feats)
    ax.set_title(title, pad=10)
    ax.grid(True, axis='x', ls='--', alpha=0.35)
    ax.set_xlim(x_min, x_max)
    ax.set_ylim(-0.5, len(feats) - 0.1)

    # 图例
    handles, labels = [], []
    for k in ['Confirmed', 'Tentative', 'Rejected']:
        if (df_long['decision'] == k).any():
            handles.append(Line2D([0], [0], lw=style_map[k]['lw'], alpha=style_map[k]['alpha'], color=color_map[k]))
            labels.append(k)
    
    if handles:
        ax.legend(handles, labels, title="Decision Type", loc='center left',
                  bbox_to_anchor=(1.02, 0.5), frameon=False)

    plt.tight_layout(rect=[0, 0, 0.86, 1])
    plt.show()

# ---------- 画图 ----------
ridgeline_plot_borutaZ(
    df_long=long_borutaZ,
    feats=feats_for_plot,
    x_label="Boruta Z-score (vs. Shadow Max)",
    title="All Selected Features: Importance vs. Shadow Maximum"
)

# ---------- 导出汇总表 ----------
borutaZ_summary = (long_borutaZ
                   .groupby(["feature","decision"])["z"]
                   .agg(['count','mean','median','std'])
                   .reset_index()
                   .sort_values("median", ascending=False)).reset_index(drop=True)

# 添加hit率统计
hit_rate = (long_borutaZ
            .groupby("feature")
            .apply(lambda x: (x["z"] > 0).mean())
            .rename("hit_rate"))

borutaZ_summary = borutaZ_summary.merge(
    hit_rate.reset_index(), 
    on="feature", 
    how="left"
)

print("\n" + "=" * 70)
print("📊 Boruta Z-score 汇总（标准方式：vs. Shadow Max）")
print("=" * 70)
print("Z > 0 表示特征重要性超过shadow最大值")
print("hit_rate = 超过shadow最大值的迭代比例")
print("=" * 70)
print(borutaZ_summary.head(30))

In [ ]:
# ===================== 第6步：基于 Boruta 选出的特征，定义模型集合 =====================

print("=" * 70)
print("定义模型集合")
print("=" * 70)

#准备好验证数据
print(f"使用特征数量: {X_train_for_models.shape[1]}")
print(f"训练集样本数: {X_train_for_models.shape[0]}")
print(f"验证集样本数: {X_val_for_models.shape[0]}")
print(f"测试集样本数: {X_test_for_models.shape[0]}")

CLASS_WEIGHT = "balanced"

# ---------- 6.1 CV 与评分器 ----------
cv_5x3 = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=RANDOM_STATE)
auc_scorer = "roc_auc"

# ---------- 6.2 各模型定义 ----------
print("\n定义模型...")

# 1) Logistic Regression
logit = LogisticRegression(
    max_iter=5000, 
    solver="lbfgs",
    class_weight=CLASS_WEIGHT,
    random_state=RANDOM_STATE
)
logit_grid = {"C": [0.0005, 0.001, 0.005]}
logit_gs = GridSearchCV(logit, logit_grid, scoring=auc_scorer,
                        cv=cv_5x3, n_jobs=-1, refit=True,
                        error_score="raise")

# 2) LASSO / Elastic Net
enet = LogisticRegression(
    penalty="elasticnet", 
    solver="saga",
    max_iter=5000,
    class_weight=CLASS_WEIGHT,
    random_state=RANDOM_STATE
)
enet_grid = {
    "l1_ratio": [0.7, 0.9, 1.0],
    "C": [0.0005, 0.001, 0.005]
}
enet_gs = GridSearchCV(enet, enet_grid, scoring=auc_scorer,
                       cv=cv_5x3, n_jobs=-1, refit=True,
                       error_score="raise")

# 3) SVM (RBF)
svm = SVC(
    kernel="rbf", 
    probability=True,
    class_weight=CLASS_WEIGHT,
    random_state=RANDOM_STATE
)
svm_grid = {
    "C": [0.1, 1, 10, 100, 1000],
    "gamma": [0.001, 0.01, 0.1, 0.5, 1.0]
}
svm_gs = GridSearchCV(svm, svm_grid, scoring=auc_scorer,
                      cv=cv_5x3, n_jobs=-1, refit=True,
                      error_score="raise")

# 4) Random Forest
rf = RandomForestClassifier(
    n_estimators=200,
    bootstrap=True,
    max_samples=0.5,
    max_leaf_nodes=30,
    max_depth=2,
    min_samples_leaf=20,
    max_features=2,
    class_weight=CLASS_WEIGHT,
    n_jobs=-1,
    random_state=RANDOM_STATE
)
rf_grid = {
    "max_features": [2, 3],
    "min_samples_leaf": [10, 15, 20],
    "min_samples_split": [10, 20]
}
rf_gs = GridSearchCV(rf, rf_grid, scoring=auc_scorer,
                     cv=cv_5x3, n_jobs=-1, refit=True,
                     error_score="raise")

# 5) XGBoost
pos = int(np.sum(train_y == 1))
neg = int(np.sum(train_y == 0))
spw = (neg / max(pos, 1))

xgb = XGBClassifier(
    tree_method="hist",
    eval_metric="logloss",
    scale_pos_weight=spw,
    random_state=RANDOM_STATE
)
xgb_grid = {
    "max_depth": [2],
    "min_child_weight": [80, 100, 150],   # 进一步增加
    "gamma": [5.0, 8.0, 10.0],            # 进一步增加
    "reg_alpha": [10.0, 15.0, 20.0],      # 进一步增加
    "reg_lambda": [10.0, 15.0, 20.0],     # 进一步增加
    "colsample_bytree": [0.2, 0.3],       # 进一步减少
    "subsample": [0.3, 0.4],              # 进一步减少
    "learning_rate": [0.005, 0.01],       # 进一步降低
    "n_estimators": [200, 300]
}
xgb_gs = GridSearchCV(xgb, xgb_grid, scoring=auc_scorer,
                      cv=cv_5x3, n_jobs=-1, refit=True,
                      error_score="raise")

# ---------- 6.3 模型字典 ----------
models = {
    "Logistic": logit_gs,
    "LASSO/ElasticNet": enet_gs,
    "SVM": svm_gs,
    "RandomForest": rf_gs,
    "XGBoost": xgb_gs
}

print(f"\n 模型定义完成")
print(f"将对以下 {len(models)} 个模型进行 5×3 CV 调参：")
for i, name in enumerate(models.keys(), 1):
    print(f"   {i}. {name}")

print("\n" + "=" * 70)

In [ ]:
# ===================== 第6.4步：辅助函数定义 =====================
# 这些函数将在第7步的模型评估中使用
def to_score(estimator, X):
    """
    从任意sklearn分类器获取正类概率分数
    兼容Pipeline、CalibratedClassifierCV、SVM等
    """
    if hasattr(estimator, 'predict_proba'):
        proba = estimator.predict_proba(X)
        if proba.ndim == 1:
            return proba
        elif proba.shape[1] >= 2:
            return proba[:, 1]
        return np.squeeze(proba)
    elif hasattr(estimator, 'decision_function'):
        # 升级：z-score + sigmoid，确保输出为概率
        scores = estimator.decision_function(X)
        mu = np.mean(scores)
        sigma = np.std(scores) + 1e-12
        return expit((scores - mu) / sigma)
    else:
        return estimator.predict(X).astype(float)


def youden_threshold(y_true, y_score):
    """
    根据Youden指数（灵敏度+特异度-1的最大值）计算最优分类阈值
    
    Parameters:
    -----------
    y_true : array-like, 真实标签
    y_score : array-like, 预测概率分数
    
    Returns:
    --------
    threshold : float, 最优阈值
    """
    fpr, tpr, thresholds = roc_curve(y_true, y_score)
    youden_index = tpr - fpr
    optimal_idx = np.argmax(youden_index)
    optimal_threshold = float(thresholds[optimal_idx])
    return optimal_threshold


def binary_metrics_at(y_true, y_score, threshold):
    """
    在给定阈值下计算二分类的所有性能指标
    
    Parameters:
    -----------
    y_true : array-like, 真实标签 (0/1)
    y_score : array-like, 预测概率分数
    threshold : float, 分类阈值
    
    Returns:
    --------
    accuracy, sensitivity, specificity, ppv, npv : float
    """
    y_true = np.asarray(y_true, dtype=int)
    y_score = np.asarray(y_score, dtype=float)
    
    # 根据阈值进行分类
    y_pred = (y_score >= threshold).astype(int)
    
    # 计算混淆矩阵的各个组成部分
    tp = np.sum((y_pred == 1) & (y_true == 1))
    tn = np.sum((y_pred == 0) & (y_true == 0))
    fp = np.sum((y_pred == 1) & (y_true == 0))
    fn = np.sum((y_pred == 0) & (y_true == 1))
    
    # 计算各项指标
    accuracy = (tp + tn) / (tp + tn + fp + fn) if (tp + tn + fp + fn) > 0 else 0.0
    sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0.0  # 灵敏度
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0.0  # 特异度
    ppv = tp / (tp + fp) if (tp + fp) > 0 else 0.0          # 阳性预测值
    npv = tn / (tn + fn) if (tn + fn) > 0 else 0.0          # 阴性预测值
    
    return float(accuracy), float(sensitivity), float(specificity), float(ppv), float(npv)

def auc_ci_bootstrap(y_true, y_score, n_boot=1000, seed=42, alpha=0.95):
    """Bootstrap 计算 AUC 置信区间"""
    y_true = np.asarray(y_true, dtype=int)
    y_score = np.asarray(y_score, dtype=float)
    
    if len(y_true) != len(y_score):
        return np.nan, np.nan
    
    rng = np.random.RandomState(seed)
    n = len(y_true)
    
    # 类别检查
    pos_idx = np.where(y_true == 1)[0]
    neg_idx = np.where(y_true == 0)[0]
    
    if len(pos_idx) < 2 or len(neg_idx) < 2 or n < 10:
        try:
            auc = roc_auc_score(y_true, y_score)
            return float(auc), float(auc)
        except:
            return np.nan, np.nan
    
    # 分层 Bootstrap
    aucs = []
    for _ in range(n_boot):
        try:
            boot_pos = rng.choice(pos_idx, size=len(pos_idx), replace=True)
            boot_neg = rng.choice(neg_idx, size=len(neg_idx), replace=True)
            boot_idx = np.concatenate([boot_pos, boot_neg])
            aucs.append(roc_auc_score(y_true[boot_idx], y_score[boot_idx]))
        except:
            continue
    
    if len(aucs) == 0:
        try:
            auc = roc_auc_score(y_true, y_score)
            return float(auc), float(auc)
        except:
            return np.nan, np.nan
    
    # 置信区间
    lower_p = ((1.0 - alpha) / 2.0) * 100
    upper_p = (alpha + (1.0 - alpha) / 2.0) * 100
    return float(np.percentile(aucs, lower_p)), float(np.percentile(aucs, upper_p))

In [ ]:
# ============================ 第7步：训练与评估 ============================

# ===================== 7.1 CV调参 =====================
fit_models = {}
cv_report = {}
grid_search_results = {}

print("=" * 70)
print("阶段1：CV调参")
print("=" * 70)

for name, est in models.items():
    try:
        est_ = clone(est)
        est_.fit(X_train_for_models, train_y)
        
        best_est = est_.best_estimator_ if hasattr(est_, "best_estimator_") else est_
        best_score = est_.best_score_ if hasattr(est_, "best_score_") else np.nan
        
        fit_models[name] = best_est
        grid_search_results[name] = est_
        cv_report[name] = best_score
        
        print(f"[✓] {name:20s} | CV AUC = {best_score:.4f}")
    except Exception as e:
        print(f"[✗] {name:20s} | 失败: {e}")
        fit_models[name] = None
        grid_search_results[name] = None
        cv_report[name] = np.nan


# ===================== 7.2 验证集选择最优模型 =====================
print("\n" + "=" * 70)
print("阶段2：验证集选择最优模型")
print("=" * 70)

val_aucs = {}
for name, est in fit_models.items():
    if est is None:
        val_aucs[name] = np.nan
        continue
    try:
        y_val_prob = to_score(est, X_val_for_models)
        val_aucs[name] = roc_auc_score(val_y, y_val_prob)
        print(f"[✓] {name:20s} | Val AUC = {val_aucs[name]:.4f}")
    except Exception as e:
        print(f"[✗] {name:20s} | 失败: {e}")
        val_aucs[name] = np.nan

best_model_name = max(val_aucs.items(), key=lambda x: x[1])[0]
best_model_params = fit_models[best_model_name]
print(f"\n🏆 最优模型: {best_model_name} (Val AUC = {val_aucs[best_model_name]:.4f})")


# ===================== 7.3 确定最优模型（不重训练） =====================
print("\n" + "=" * 70)
print("阶段3：确定最优模型")
print("=" * 70)

# 最优模型直接使用 7.1 训练好的
best_model = fit_models[best_model_name]

# 校准（只用训练集）
calibrated_best = CalibratedClassifierCV(estimator=clone(best_model), method="sigmoid", cv=5)
calibrated_best.fit(X_train_for_models, train_y)

# 阈值（只用训练集）
y_train_prob = to_score(calibrated_best, X_train_for_models)
optimal_threshold = youden_threshold(train_y, y_train_prob)

print(f" 最优模型: {best_model_name}")
print(f" Youden阈值 = {optimal_threshold:.4f}")

# ===================== 7.4 所有模型评估（Train/Val/Test 独立） =====================
print("\n" + "=" * 70)
print("阶段6：所有模型评估")
print("=" * 70)
calibrated_models_all = {}
all_results = []

for name, model in fit_models.items():
    if model is None:
        continue
    
    try:
        calibrated = CalibratedClassifierCV(estimator=clone(model), method="sigmoid", cv=5)
        calibrated.fit(X_train_for_models, train_y)
        calibrated_models_all[name] = calibrated
    except:
        calibrated = model
    
    try:
        train_prob_for_thr = to_score(calibrated, X_train_for_models)
        threshold = youden_threshold(train_y, train_prob_for_thr)
    except:
        threshold = 0.5
    
    datasets = [
        ("Train", X_train_for_models, train_y),
        ("Val", X_val_for_models, val_y),
        ("Test", X_test_for_models, test_y)
    ]
    
    for split_name, Xs, yss in datasets:
        try:
            prob = to_score(calibrated, Xs)
            auc_val = roc_auc_score(yss, prob)
            ci_lo, ci_hi = auc_ci_bootstrap(yss, prob, n_boot=800, seed=RANDOM_STATE)
            acc, sen, spe, ppv, npv = binary_metrics_at(yss, prob, threshold)
            
            # 计算F1分数
            y_pred = (prob >= threshold).astype(int)
            f1 = f1_score(yss, y_pred)
            
            all_results.append({
                "Model": name, "Dataset": split_name, "AUC": auc_val,
                "AUC 95%CI": f"[{ci_lo:.3f}, {ci_hi:.3f}]", "Threshold": threshold,
                "Accuracy": acc, "Precision": ppv, "Recall": sen,  # PPV=Precision, Sensitivity=Recall
                "F1 Score": f1, "Specificity": spe, "NPV": npv
            })
        except Exception as e:
            print(f"[✗] {name} @ {split_name}: {e}")

df_all_results = pd.DataFrame(all_results)

# AUC 宽格式对比
pivot_auc = df_all_results.pivot(index='Model', columns='Dataset', values='AUC')
pivot_auc = pivot_auc[['Train', 'Val', 'Test']]
pivot_auc['Train-Test Gap'] = pivot_auc['Train'] - pivot_auc['Test']
pivot_auc = pivot_auc.sort_values('Test', ascending=False)

print("\n AUC 对比:")
display(pivot_auc.round(4))

# 所有模型三集详细性能
print("\n 所有模型完整性能详情:")
df_display = df_all_results.copy()
df_display = df_display.sort_values(['Model', 'Dataset'], 
                                     key=lambda x: x.map({'Train': 0, 'Val': 1, 'Test': 2}) if x.name == 'Dataset' else x)
df_display = df_display[['Model', 'Dataset', 'AUC', 'Accuracy', 'AUC 95%CI', 'Threshold', 
                          'Precision', 'Recall', 'F1 Score', 'Specificity', 'NPV']]
display(df_display.round(4))

# 最优模型
df_test = df_all_results[df_all_results['Dataset'] == 'Test'].sort_values('AUC', ascending=False)
best_model_name = df_test.iloc[0]['Model']
print(f"\n 最优模型: {best_model_name} (Test AUC = {df_test.iloc[0]['AUC']:.4f})")


# ===================== 7.5 拟合程度可视化 =====================
print("\n" + "=" * 70)
print("阶段7：拟合程度可视化")
print("=" * 70)

import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(12, 5))  # 改为2列

# --- 图1: 分组柱状图 ---
ax1 = axes[0]
models_order = pivot_auc.index.tolist()
x = np.arange(len(models_order))
width = 0.25

train_vals = [pivot_auc.loc[m, 'Train'] for m in models_order]
val_vals = [pivot_auc.loc[m, 'Val'] for m in models_order]
test_vals = [pivot_auc.loc[m, 'Test'] for m in models_order]

bars1 = ax1.bar(x - width, train_vals, width, label='Train', color='#3498db', alpha=0.8)
bars2 = ax1.bar(x, val_vals, width, label='Val', color='#2ecc71', alpha=0.8)
bars3 = ax1.bar(x + width, test_vals, width, label='Test', color='#e74c3c', alpha=0.8)

ax1.set_ylabel('AUC', fontsize=12)
ax1.set_title('Train / Val / Test AUC Comparison', fontsize=13, fontweight='bold')
ax1.set_xticks(x)
ax1.set_xticklabels(models_order, rotation=30, ha='right')
ax1.legend(loc='lower right')
ax1.set_ylim(0.5, 1.0)
ax1.axhline(y=0.7, color='gray', linestyle='--', alpha=0.5, label='AUC=0.7')
ax1.grid(axis='y', alpha=0.3)

for bars in [bars1, bars2, bars3]:
    for bar in bars:
        height = bar.get_height()
        ax1.annotate(f'{height:.3f}', xy=(bar.get_x() + bar.get_width()/2, height),
                     xytext=(0, 3), textcoords="offset points", ha='center', va='bottom', fontsize=8)

# --- 图2: Train-Test Gap 柱状图 ---
ax2 = axes[1]
gaps = [pivot_auc.loc[m, 'Train-Test Gap'] for m in models_order]
colors = ['#27ae60' if g < 0.05 else '#f39c12' if g < 0.10 else '#e67e22' if g < 0.15 else '#c0392b' for g in gaps]

bars_gap = ax2.bar(x, gaps, width=0.5, color=colors, alpha=0.85, edgecolor='black', linewidth=0.5)

ax2.set_ylabel('Train - Test Gap', fontsize=12)
ax2.set_title('Overfitting Assessment (Train-Test Gap)', fontsize=13, fontweight='bold')
ax2.set_xticks(x)
ax2.set_xticklabels(models_order, rotation=30, ha='right')
ax2.axhline(y=0.05, color='green', linestyle='--', alpha=0.7, linewidth=1.5, label='Good (<0.05)')
ax2.axhline(y=0.10, color='orange', linestyle='--', alpha=0.7, linewidth=1.5, label='Acceptable (<0.10)')
ax2.axhline(y=0.15, color='red', linestyle='--', alpha=0.7, linewidth=1.5, label='Concern (>0.15)')
ax2.legend(loc='upper right', fontsize=9)
ax2.set_ylim(0, max(gaps) * 1.3)
ax2.grid(axis='y', alpha=0.3)

for bar, gap in zip(bars_gap, gaps):
    ax2.annotate(f'{gap:.4f}', xy=(bar.get_x() + bar.get_width()/2, bar.get_height()),
                 xytext=(0, 3), textcoords="offset points", ha='center', va='bottom', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n 可视化已保存: overfitting_analysis.png")

# ===================== 7.6 创建后续变量 & 保存模型 =====================
print("\n" + "=" * 70)
print("阶段8：保存模型")
print("=" * 70)

# 使用 7.3 中创建的校准模型（不再引用 calibrated_final）
# calibrated_best 已在 7.3 中定义
best_est = fit_models[best_model_name]
calibrated_models = {best_model_name: calibrated_best}
y_test_prob = to_score(calibrated_best, X_test_for_models)
thr_star = optimal_threshold
has_val = True  # 你有验证集

# 保存模型
import pickle
test_auc_value = df_test[df_test['Model']==best_model_name]['AUC'].values[0]

# 获取最优参数
if grid_search_results.get(best_model_name) is not None:
    best_params = grid_search_results[best_model_name].best_params_
else:
    best_params = None

model_artifacts = {
    'model': calibrated_best,
    'model_name': best_model_name,
    'features': list(X_train_for_models.columns),
    'threshold': optimal_threshold,
    'best_params': best_params,
    'metadata': {'train_size': len(X_train_for_models), 'test_auc': test_auc_value, 'random_state': RANDOM_STATE}
}

with open("best_model_complete.pkl", 'wb') as f:
    pickle.dump(model_artifacts, f)

print(f" 模型已保存: best_model_complete.pkl")
print(f" 最优模型: {best_model_name}")
print(f" 阈值: {optimal_threshold:.4f} | 测试AUC: {test_auc_value:.4f}")
print("=" * 70)

In [ ]:
# ===================== 7.7 ROC 曲线（所有模型） =====================
def plot_roc_panel(models_dict):
    # 展示 Train, Val, Test 三个数据集
    sets = [
        ("Train", X_train_for_models, train_y),
        ("Val", X_val_for_models, val_y),
        ("Test", X_test_for_models, test_y)
    ]

    fig, axes = plt.subplots(1, 3, figsize=(18, 5))

    for ax, (set_name, Xs, ys) in zip(axes, sets):
        for name, est in models_dict.items():
            y_prob = to_score(est, Xs)
            fpr, tpr, _ = roc_curve(ys, y_prob)
            auc_val = roc_auc_score(ys, y_prob)
            ax.plot(fpr, tpr, lw=2, label=f"{name} (AUC={auc_val:.3f})")

        ax.plot([0, 1], [0, 1], 'k--', lw=1)
        ax.set_title(f"ROC — {set_name}", fontsize=13)
        ax.set_xlabel("False Positive Rate")
        ax.set_ylabel("True Positive Rate")
        ax.legend(loc="lower right", fontsize=9)
        ax.grid(alpha=0.3)

    fig.suptitle("ROC Curves by Dataset (All Calibrated Models)", y=1.02, fontsize=15)
    plt.tight_layout()
    plt.savefig('roc_curves_all_models.png', dpi=150, bbox_inches='tight')
    plt.show()

plot_roc_panel(calibrated_models_all)

# ===================== 7.8 DCA（训练集、测试集） =====================
def decision_curve(y_true, y_prob, thresholds=None):
    y = np.asarray(y_true).astype(int)
    p = np.asarray(y_prob).astype(float)
    n = len(y)
    prev = y.mean()
    if thresholds is None:
        thresholds = np.linspace(0.01, 0.60, 30)
    out = []
    for pt in thresholds:
        pred = (p >= pt).astype(int)
        tp = np.sum((pred == 1) & (y == 1))
        fp = np.sum((pred == 1) & (y == 0))
        nb_model = (tp / n) - (fp / n) * (pt / (1 - pt))
        nb_all = prev - (1 - prev) * (pt / (1 - pt))
        nb_none = 0.0
        out.append((pt, nb_model, nb_all, nb_none))
    return pd.DataFrame(out, columns=["pt", "NB_model", "NB_all", "NB_none"])

def plot_dca(df, title="", mark_pt=None, db_name="MIMIC-IV"):
    fig, ax = plt.subplots(figsize=(8, 6))
    ax.plot(df["pt"], df["NB_model"], lw=2, label="Model")
    ax.plot(df["pt"], df["NB_all"], lw=2, color="gray", label="Treat All")
    ax.plot(df["pt"], df["NB_none"], lw=2, ls="--", color="gray", label="Treat None")

    if mark_pt is not None:
        row = df.iloc[(df["pt"] - mark_pt).abs().argmin()]
        ax.scatter([row["pt"]], [row["NB_model"]], s=45, label=f"thr≈{row['pt']:.2f}", color="red")

    ax.set_xlabel("Threshold Probability")
    ax.set_ylabel("Net Benefit")
    ax.set_title(f"{title} — {db_name}")
    ax.legend()
    ax.grid(alpha=0.3)
    
    # 固定 y 轴范围
    ax.set_ylim(-0.05, 0.30)
    
    plt.tight_layout()
    plt.show()

# 计算训练集的预测概率（用于 DCA）
y_train_prob = to_score(calibrated_best, X_train_for_models)

# 训练集 DCA
plot_dca(decision_curve(train_y, y_train_prob),
         title=f"DCA — Calibrated {best_model_name}", mark_pt=thr_star, db_name="Train")

# 测试集 DCA
plot_dca(decision_curve(test_y, y_test_prob),
         title=f"DCA — Calibrated {best_model_name}", mark_pt=thr_star, db_name="Test")

# 找到蓝线穿过0的精确阈值
dca_test = decision_curve(test_y, y_test_prob)
valid_range = dca_test[dca_test['NB_model'] > 0]['pt']
print(f"模型净收益>0的阈值范围: {valid_range.min():.2f} - {valid_range.max():.2f}")

In [ ]:
# ===================== 第8步：SHAP 解释分析 =====================
print("=" * 70)
print("第8步：SHAP 解释分析")
print("=" * 70)

# ---------------- 8.1 准备工作：使用Boruta筛选后的特征 ----------------
features_for_shap = list(X_train_for_models.columns)
print(f"\n✅ Boruta筛选特征数: {len(features_for_shap)}")
print(f"   - 确认特征: {len(selected_features)}")
print(f"   - 待定特征: {len(tentative_features)}")

# 获取最优模型（已校准）
shap_model = calibrated_best
print(f"\n✅ SHAP 解释模型: {best_model_name} (已校准)")

# ---------------- 8.2 构建 SHAP Explainer ----------------
print("\n" + "=" * 70)
print("构建 SHAP Explainer")
print("=" * 70)

# 背景数据：从训练集中采样
rng = np.random.RandomState(RANDOM_STATE)
n_background = min(200, len(X_train_for_models))
bg_idx = rng.choice(len(X_train_for_models), size=n_background, replace=False)
X_background = X_train_for_models.iloc[bg_idx].copy()

print(f"背景样本数: {n_background} (来自训练集)")

# 预测函数
def predict_proba_class1(X):
    """SHAP专用：返回正类概率"""
    return to_score(shap_model, X)

# 创建 Explainer
try:
    explainer = shap.Explainer(predict_proba_class1, X_background)
    print("✅ SHAP Explainer 创建成功")
except Exception as e:
    print(f"⚠️ 使用函数式 Explainer 失败: {e}")
    print("   尝试使用 KernelExplainer...")
    explainer = shap.KernelExplainer(predict_proba_class1, X_background)
    print("✅ SHAP KernelExplainer 创建成功")

# ---------------- 8.2.1 计算测试集的 SHAP 值 ----------------
print("\n" + "=" * 70)
print("计算 SHAP 值")
print("=" * 70)

# 只计算测试集 SHAP
print(f"计算测试集 SHAP 值 (n={len(X_test_for_models)})...")
sv_test = explainer(X_test_for_models)

# 提取数组
def extract_shap_array(sv):
    if hasattr(sv, 'values'):
        arr = sv.values
    else:
        arr = sv
    if arr.ndim == 3:
        arr = arr[:, :, 1]
    return arr

shap_array_test = extract_shap_array(sv_test)
print(f"✅ 测试集 SHAP 值: {shap_array_test.shape}")

# 统一变量名
sv = sv_test
X_explain = X_test_for_models
shap_values = sv_test
X_shap = X_test_for_models
y_shap = test_y
shap_array = shap_array_test
which_split = "Test"

# ---------------- 8.3 全局特征重要性：Beeswarm 图 ----------------
print("\n" + "=" * 70)
print("8.3 全局特征重要性可视化")
print("=" * 70)

plt.figure(figsize=(10, max(6, 0.35 * len(features_for_shap))))
shap.summary_plot(
    shap_values,
    features=X_shap,
    feature_names=features_for_shap,
    show=False,
    max_display=len(features_for_shap)
)
plt.title(f"SHAP Summary Plot — {best_model_name} (Critically Ill Patients)", 
          fontsize=14, pad=15, fontweight='bold')
plt.tight_layout()
plt.show()

# ---------------- 8.4 Dependence Plots（Top 9 特征） ----------------
print("\n" + "=" * 70)
print("8.4 SHAP Dependence Plots")
print("=" * 70)

# 计算特征重要性并选择 Top 9
mean_abs_shap = np.abs(shap_array).mean(axis=0)
importance_df = pd.Series(mean_abs_shap, index=features_for_shap).sort_values(ascending=False)

n_top = min(15, len(features_for_shap))
top_features = importance_df.head(n_top).index.tolist()

print(f"绘制 Top {n_top} 特征的依赖图:")
for i, feat in enumerate(top_features, 1):
    print(f"  {i}. {feat} (Mean |SHAP|={importance_df[feat]:.4f})")

# 布局
ncols = 3
nrows = int(np.ceil(n_top / ncols))
fig, axes = plt.subplots(nrows, ncols, figsize=(15, 5 * nrows))
axes = axes.flatten() if n_top > 1 else [axes]

for idx, feature in enumerate(top_features):
    ax = axes[idx]
    feat_idx = features_for_shap.index(feature)
    x_values = X_shap[feature].values
    y_values = shap_array[:, feat_idx]
    
    valid_mask = np.isfinite(x_values) & np.isfinite(y_values)
    x_values = x_values[valid_mask]
    y_values = y_values[valid_mask]
    
    if len(x_values) > 0:
        vmin_feat = np.percentile(x_values, 1)
        vmax_feat = np.percentile(x_values, 99)
        norm_feat = mpl.colors.Normalize(vmin=vmin_feat, vmax=vmax_feat)
        
        scatter = ax.scatter(
            x_values, y_values,
            c=x_values,
            cmap='RdYlBu_r',
            norm=norm_feat,
            s=25,
            alpha=0.6,
            edgecolors='none'
        )
        
        cbar_ax = inset_axes(
            ax,
            width="3%",
            height="80%",
            loc='center right',
            bbox_to_anchor=(0.18, 0, 1, 1),
            bbox_transform=ax.transAxes,
            borderpad=0
        )
        plt.colorbar(scatter, cax=cbar_ax)
        
        ax.set_xlabel(feature, fontsize=10)
        ax.set_ylabel('SHAP value', fontsize=10)
        ax.set_title(f'Dependence: {feature}', fontsize=11, fontweight='bold')
        ax.grid(alpha=0.3, linestyle='--')
        ax.axhline(0, color='black', linewidth=0.8, linestyle='-', alpha=0.3)

for idx in range(n_top, len(axes)):
    axes[idx].axis('off')

fig.suptitle(f'SHAP Dependence Plots — {best_model_name} (Critically Ill Patients)',
             y=1.00, fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

# ---------------- 8.5 个体解释：Waterfall 图 ----------------
print("\n" + "=" * 70)
print("8.5 个体解释（Waterfall Plots）")
print("=" * 70)

y_pred_proba = predict_proba_class1(X_shap)
idx_high = np.argmax(y_pred_proba)
idx_low = np.argmin(y_pred_proba)

print(f"最高死亡风险样本: 索引={idx_high}, 预测概率={y_pred_proba[idx_high]:.4f}")
print(f"最低死亡风险样本: 索引={idx_low}, 预测概率={y_pred_proba[idx_low]:.4f}")

def plot_waterfall_native(shap_vals, X_data, sample_idx, title, shap_arr, max_display=15):
    try:
        if hasattr(shap_vals, 'base_values'):
            base_value = shap_vals.base_values[sample_idx]
        else:
            base_value = explainer.expected_value
            if isinstance(base_value, (list, np.ndarray)):
                base_value = base_value[1] if len(base_value) > 1 else base_value[0]
        
        explanation = shap.Explanation(
            values=shap_arr[sample_idx],
            base_values=base_value,
            data=X_data.iloc[sample_idx].values,
            feature_names=features_for_shap
        )
        
        plt.rcParams['font.family'] = 'DejaVu Sans'
        plt.rcParams['font.size'] = 10
        plt.rcParams['axes.unicode_minus'] = False
        
        fig = plt.figure(figsize=(12, max(10, 0.5 * max_display)))
        shap.plots.waterfall(explanation, max_display=max_display, show=False)
        plt.gcf().suptitle(title, fontsize=14, fontweight='bold', y=0.98)
        plt.tight_layout()
        plt.show()
        
        print(f"\n该样本的详细特征值:")
        sample_data = X_data.iloc[sample_idx]
        shap_vals_sample = shap_arr[sample_idx]
        
        abs_shap = np.abs(shap_vals_sample)
        sorted_idx = np.argsort(abs_shap)[::-1][:max_display]
        
        print(f"{'特征名':<25} {'特征值':>12} {'SHAP值':>12}")
        print("-" * 52)
        for idx in sorted_idx:
            feat_name = features_for_shap[idx]
            feat_val = sample_data[feat_name]
            shap_val = shap_vals_sample[idx]
            print(f"{feat_name:<25} {feat_val:>12.4f} {shap_val:>12.4f}")
        
    except Exception as e:
        print(f"⚠️ Waterfall 图绘制失败: {e}")
        import traceback
        traceback.print_exc()

print("\n" + "=" * 70)
print("高死亡风险样本 Waterfall 图")
print("=" * 70)
plot_waterfall_native(
    shap_values, X_shap, idx_high,
    f"High Mortality Risk Patient (Prob={y_pred_proba[idx_high]:.3f}) — {best_model_name}",
    shap_array,
    max_display=15
)

print("\n" + "=" * 70)
print("低死亡风险样本 Waterfall 图")
print("=" * 70)
plot_waterfall_native(
    shap_values, X_shap, idx_low,
    f"Low Mortality Risk Patient (Prob={y_pred_proba[idx_low]:.3f}) — {best_model_name}",
    shap_array,
    max_display=15
)

print("\n✅ 个体解释完成")

# ---------------- 8.6 特征重要性汇总表 ----------------
print("\n" + "=" * 70)
print("8.6 特征重要性汇总")
print("=" * 70)

summary_df = pd.DataFrame({
    'Feature': features_for_shap,
    'Mean_|SHAP|': np.abs(shap_array).mean(axis=0),
    'Std_SHAP': shap_array.std(axis=0),
    'Mean_SHAP': shap_array.mean(axis=0)
})

summary_df = summary_df.sort_values('Mean_|SHAP|', ascending=False).reset_index(drop=True)
summary_df['Rank'] = range(1, len(summary_df) + 1)

print(f"\n特征重要性排序（基于测试集，n={len(X_shap)}）:")
display(summary_df[['Rank', 'Feature', 'Mean_|SHAP|', 'Mean_SHAP', 'Std_SHAP']].round(4))


In [ ]:
# ======== Force-like 多样本折线图（稳健版，适配校准后模型） ========
def _proba_from_any(est, X_df):
    """
    从任意二分类估计器拿“正类概率”样式分数。
    兼容 CalibratedClassifierCV / Pipeline / SVM(decision_function) / 只有 predict 的模型。
    """
    if hasattr(est, "predict_proba"):
        p = est.predict_proba(X_df)
        return p[:, 1] if p.ndim == 2 and p.shape[1] >= 2 else np.squeeze(p).astype(float)
    if hasattr(est, "decision_function"):
        s = np.asarray(est.decision_function(X_df), dtype=float)
        # z-score 后过 sigmoid，避免尺度差异
        mu, sd = float(np.mean(s)), float(np.std(s) + 1e-12)
        from scipy.special import expit
        return expit((s - mu) / sd)
    pred = np.squeeze(est.predict(X_df)).astype(float)
    if set(np.unique(pred)).issubset({0.0, 1.0}):
        eps = 1e-6
        return pred * (1 - 2 * eps) + eps
    # 最后兜底：min-max 到(0,1)
    pmin, pmax = np.min(pred), np.max(pred)
    return (pred - pmin) / (pmax - pmin + 1e-12)


def plot_force_like_lines(
    sv, X_df, title="",
    top_n=10,
    smooth=0,                # 滑动窗口平滑（>1 开启）
    legend_out=True,
    order_scores=None,       # 若提供则按概率由高到低排序
    fixed_top_idx=None,      # 若提供则强制使用训练得到的列索引
    ylim=None,
    max_legend=10,           # 图例最多显示多少条
    clip_pct=(0.5, 99.5)     # y 按分位裁剪，抑制极端点
):
    """
    用折线方式复现 Fig.6 风格的“多样本 force plot”（静态 PNG）。
    重要：sv 必须与 X_df 的行列完全一致（同样本、同列顺序）。
    """
    # ----- 1) 取 SHAP 数组（正类） -----
    sv_values = sv.values if hasattr(sv, "values") else sv
    sv_values = np.asarray(sv_values)
    if sv_values.ndim == 3:
        sv_values = sv_values[..., 1]

    # 安全检查
    n_samples, n_feats = sv_values.shape
    feat_cols = np.array(X_df.columns.tolist())
    assert n_feats == len(feat_cols), \
        f"sv 列数({n_feats})与 X_df 列数({len(feat_cols)})不一致；请保证与 explainer 输入一致。"
    assert n_samples == len(X_df), \
        f"sv 行数({n_samples})与 X_df({len(X_df)})不一致；不要在解释后再重排/抽样。"

    # ----- 2) 样本排序 -----
    if order_scores is not None:
        order_scores = np.asarray(order_scores)
        assert order_scores.shape[0] == n_samples, "order_scores 与样本数不一致。"
        order = np.argsort(order_scores)[::-1]  # 概率从高到低
    else:
        order = np.argsort(np.sum(np.abs(sv_values), axis=1))[::-1]  # |SHAP| 总和
    S = sv_values[order, :]

    # ----- 3) 选前 top_n 特征（或使用固定索引） -----
    top_n = int(max(1, min(top_n, n_feats)))
    if fixed_top_idx is None:
        mean_abs = np.nanmean(np.abs(S), axis=0)
        top_idx = np.argsort(mean_abs)[-top_n:][::-1]
    else:
        top_idx = np.array(fixed_top_idx, dtype=int)
        top_idx = top_idx[(top_idx >= 0) & (top_idx < n_feats)]
        if len(top_idx) == 0:
            raise ValueError("fixed_top_idx 全部越界。")
    feat_names = feat_cols[top_idx].tolist()

    # ----- 4) 配色：高饱和对比 -----
    palette = [
        "#e41a1c", "#377eb8", "#4daf4a", "#984ea3", "#ff7f00",
        "#ffff33", "#a65628", "#f781bf", "#999999", "#66c2a5",
        "#1b9e77", "#d95f02", "#7570b3"  # 以防 top_n > 10
    ]
    colors = [palette[i % len(palette)] for i in range(len(top_idx))]

    # ----- 5) 绘图 -----
    plt.figure(figsize=(10, 5))
    x = np.arange(S.shape[0])

    # y 轴分位裁剪范围预先算好（便于两次调用保持一致）
    if ylim is None and clip_pct is not None:
        y_all = S[:, top_idx].ravel()
        y_all = y_all[np.isfinite(y_all)]
        if y_all.size > 20:
            lo, hi = np.nanpercentile(y_all, list(clip_pct))
            ylim = (float(lo), float(hi))

    for k, (c, j, name) in enumerate(zip(colors, top_idx, feat_names)):
        y = S[:, j].astype(float)
        if ylim is not None:
            y = np.clip(y, ylim[0], ylim[1])
        if smooth and smooth > 1:
            y = pd.Series(y).rolling(int(smooth), min_periods=1, center=True).mean().values
        lw = 1.8 if k < max_legend else 1.2  # 多余线条略细
        plt.plot(x, y, color=c, lw=lw, alpha=0.95, label=name if k < max_legend else None)

    plt.axhline(0, color="black", lw=1.0)
    plt.xlim(0, S.shape[0])
    if ylim is not None:
        # 再稍微留白，视觉不贴边
        pad = 0.03 * (ylim[1] - ylim[0])
        plt.ylim(ylim[0] - pad, ylim[1] + pad)

    plt.xlabel("Pneumonia Patients", fontsize=11)
    plt.ylabel("SHAP value contribution", fontsize=11)
    if title:
        plt.title(title, fontsize=13, pad=10, fontweight="bold")

    # ----- 6) 图例：控制数量，避免溢出 -----
    if legend_out:
        plt.legend(
            title="Top features",
            loc="center left",
            bbox_to_anchor=(1.03, 0.5),
            frameon=False,
            fontsize=9.5,
            title_fontsize=10.5
        )
        plt.tight_layout(rect=[0, 0, 0.86, 1])
    else:
        plt.legend(frameon=False, fontsize=9)
        plt.tight_layout()

    plt.show()
    return top_idx


# —— 选择用于生成概率的“最终模型”：优先校准后的 —— 
_final_est_for_plot = calibrated_best if ('calibrated_best' in globals()) else best_est

# —— 训练集：按模型“概率”排序，返回 top_idx —— 
train_scores = _proba_from_any(_final_est_for_plot, train_X_imp.loc[:, selected_all])
top_idx = plot_force_like_lines(
    sv_train,
    train_X_imp.loc[:, selected_all],
    title=f"Force-like plot — MIMIC-IV (Train){' (Calibrated)' if 'calibrated_best' in globals() else ''}",
    top_n=10,
    smooth=5,
    order_scores=train_scores
)

# —— 测试集：用训练阶段的 top_idx；保持 y 轴一致便于对比 —— 
test_scores = _proba_from_any(_final_est_for_plot, test_X_imp.loc[:, selected_all])

# 统一 y 轴范围：基于 train/test 两边的 sv 共同决定（稳）
_sv_train_arr = sv_train.values if hasattr(sv_train, "values") else sv_train
if _sv_train_arr.ndim == 3: _sv_train_arr = _sv_train_arr[..., 1]
_sv_test_arr  = sv_test.values  if hasattr(sv_test, "values")  else sv_test
if _sv_test_arr.ndim  == 3: _sv_test_arr  = _sv_test_arr[..., 1]

y_abs_max = np.nanmax(np.abs(np.concatenate([_sv_train_arr[:, top_idx], _sv_test_arr[:, top_idx]], axis=0)))
# 给一点余量
ylim_shared = (-1.05 * y_abs_max, 1.05 * y_abs_max)

_ = plot_force_like_lines(
    sv_test,
    test_X_imp.loc[:, selected_all],
    title=f"Force-like plot — MIMIC-IV (Test){' (Calibrated)' if 'calibrated_best' in globals() else ''}",
    smooth=5,
    order_scores=test_scores,
    fixed_top_idx=top_idx,
    ylim=ylim_shared
)
